In [7]:
import json
from deep_translator import GoogleTranslator


In [10]:
# Initialize translator with Urdu as target language
translator = GoogleTranslator(source='auto', target='ur')  # 'ur' for Urdu

# Define file paths
input_file = "fever_chunk.jsonl"
output_file = "translated.jsonl"

# Open input and output files
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)  # Load JSON object from line
        
        if "text" in data:  # Assuming the text to translate is in the "text" key
            # Translate the text
            data["text"] = translator.translate(data["text"])
        
        # Write back as JSON
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print(f"Translation completed! Check {output_file}")

Translation completed! Check translated.jsonl
